In [1]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [2]:
stopwords = stopwords.words("english")

print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [3]:
temp=pd.read_csv('relv.csv')
df = temp
df['keywords'] = df['wosarticle__de']+ " " +  df['wosarticle__wc']
df['content'] = df['title']+ " "+ df['content']+ " " +df['keywords']

In [4]:
data = df['content'].to_numpy()

ids = pd.DataFrame(df['id'], columns = ['id'])


In [5]:
import en_core_web_sm

nlp = en_core_web_sm.load()
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0])

evaluation performance base analysis low storey building type structure generally carry building isolate single stand building interaction adjacent building incorporate interaction adjacent building seismic performance risk retrofitting analysis use proper modelling technique seem important mean obtain deviation result conventional analysis result isolated single model structure literature review carry follow topic adjacent building model push analysis earthquake acceleration time series database analysis scaled record time history analysis equation motion beta method fragility curve build performance level aim study evaluation vulnerability risk fragility curve seismic performance adjacent building interact structure result show interact isolate analysis type yield different seismic performance level analysis scale accelogram fragility curve time history analysis adjacent building interaction none analysis scale accelogram fragility curve time history analysis adjacent building intera

In [7]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['evaluation', 'performance', 'base', 'analysis', 'low', 'storey', 'building', 'type', 'structure', 'generally', 'carry', 'building', 'isolate', 'single', 'stand', 'building', 'interaction', 'adjacent', 'building', 'incorporate']


In [8]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=4, threshold=30) 
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=40)  

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams (texts):
    return ([bigram[doc] for doc in texts])

def make_trigrams (texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[100])

['development', 'different', 'base', 'different', 'base', 'dynamic', 'study', 'aim', 'develop', 'fragility', 'curve', 'reinforce_concrete_building', 'different', 'height', 'base', 'dynamic', 'analysis', 'model', 'moment_resist', 'concrete', 'frame', 'use', 'storey', 'frame', 'design', 'base', 'sap', 'program', 'use', 'main', 'analysis', 'tool', 'obtain', 'limit_state', 'then', 'incremental_dynamic_analysis', 'carry', 'ground_motion_record', 'obtain', 'curve', 'compare', 'limit_state', 'maximum', 'allowable', 'drift', 'calculation', 'maximum', 'allowable', 'drift', 'base', 'formula', 'result', 'storey', 'well', 'structural', 'performance', 'same', 'ground_motion', 'compare', 'other', 'storey', 'curve', 'develop', 'consider', 'result', 'parameter', 'base', 'result', 'fragility', 'curve', 'high', 'probability', 'exceed', 'limit_state', 'storey', 'storey', 'low', 'probability', 'author_exclusive_license', 'limit', 'curve', 'motion', 'incremental_dynamic_analysis', 'reinforce_concrete', 'no

In [9]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)
 
texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value =0.035

drop_words = []
words_missing_in_tfidf = []
low_value_words = []
rek_words =['building','earthquake','fragility', 'seismic','vulnerability','uncertainty','reinforce_concrete_building','empirical','residential','concrete','masonry', 'nonlinear','case_study','region','numerical','reinforce_concrete_frame','country','statistical','disaster','analytical','ground_motion','regression','shear','material','prediction','reinforce_concrete','nonlinear_time_history','incremental_dynamic_analysis','historical','steel','construction_material','flooding','pushover','local','safety','depth','nonlinear_static','resilience','infrastructure','acceleration','resilient','hospital','low_rise','urban','mechanical','flood','coastal','traditional','wall','software','time_history','human_casualty','hybrid','experiment','preparedness','expert','error','inundation_depth','italian']

for i in range(0, len(corpus)):
    bow = corpus[i]
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        if id2word[item] not in drop_words:
            drop_words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    #new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    #corpus[i] = new_bow

for i in range(0, len(corpus)):
    bow = corpus[i]
    new_bow = []
    for b in bow:
        if (id2word[b[0]] in rek_words) or (id2word[b[0]] not in drop_words):
            new_bow.append(b)
    corpus[i] = new_bow
    

In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_bigrams_trigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3888993123014293


In [12]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=20)


C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [10]:
print (lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.162*"ground_motion" + 0.109*"reinforce_concrete_frame" + 0.092*"high_rise" + 0.088*"infill" + 0.063*"sequence" + 0.031*"reinforce" + 0.028*"beam" + 0.025*"strong_motion" + 0.022*"aftershock" + 0.017*"mainshock"'), (1, '0.265*"wall" + 0.132*"unreinforced_masonry" + 0.080*"partition" + 0.077*"resistant" + 0.019*"inadequate" + 0.018*"portion" + 0.012*"thorough" + 0.009*"pressing" + 0.008*"reality" + 0.006*"failure_mode"'), (2, '0.311*"building" + 0.263*"earthquake" + 0.017*"low_rise" + 0.015*"acceleration" + 0.011*"safety" + 0.011*"precast" + 0.010*"liquefaction" + 0.010*"region" + 0.008*"disaster" + 0.007*"time_history"'), (3, '0.230*"vulnerability" + 0.124*"building" + 0.098*"seismic" + 0.052*"masonry" + 0.040*"empirical" + 0.023*"earthquake" + 0.012*"region" + 0.012*"urban" + 0.012*"unreinforced_masonry_building" + 0.012*"country"'), (4, '0.000*"ultrasonic_device" + 0.000*"pseudo" + 0.000*"sustained" + 0.000*"repository" + 0.000*"publicly" + 0.000*"protocol" + 0.000*"move_avera

In [13]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_bigrams_trigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.310741735201438

Coherence Score:  0.41705248726440824


In [14]:
topic_var = pd.DataFrame(0,index=np.arange(len(corpus)),columns = ['id','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','Topic','Keywords'])
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    for i, row in enumerate(ldamodel[corpus]):
        topic_var['id'][i] = ids['id'][i]
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            col = str(topic_num)
            topic_var[col][i]=round(prop_topic,4)
            if j == 0:
                topic_var['Topic'][i] = str(topic_num)
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                topic_var['Keywords'][i] = topic_keywords
                
format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_bigrams_trigrams)



C:\Users\User\AppData\Local\Temp\ipykernel_14548\2113404940.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_var['Topic'][i] = str(topic_num)
C:\Users\User\AppData\Local\Temp\ipykernel_14548\2113404940.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_var['Keywords'][i] = topic_keywords
C:\Users\User\AppData\Local\Temp\ipykernel_14548\2113404940.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_var[col][i]=ro

In [15]:
topic_var.to_csv('TopicVar.csv', index=False)

In [28]:
methoR=pd.read_csv('RelevantMethod.csv')

df_merged = pd.merge(topic_var, methoR, left_on='id', right_on='id', how='right')

In [29]:
df_merged2 = df_merged.drop(['content','title','1 - Analytical - prediction','1 - Empirical - prediction','1 - Hybrid - prediction','1 - ML - prediction'], axis=1)

In [30]:
df_merged2.to_csv('ToTSNE.csv', index=False)

In [31]:
Main_topics = pd.DataFrame(columns=['id','Earthquake','Flood','Tsunami','Hurricane','RC','Wood','Steel','Masonry','Lowrise','Medrise','Regional','Urban','Residential'])

Main_topics['id'] = topic_var['id']
Main_topics['Earthquake'] = topic_var['17']
Main_topics['Flood'] = topic_var['9']
Main_topics['Tsunami'] = topic_var['11']
Main_topics['Hurricane'] = topic_var['4']+topic_var['7']+0.83*topic_var['3']
Main_topics['RC'] = topic_var['0']+ 0.07*topic_var['17']
Main_topics['Wood'] = topic_var['3']+ topic_var['8']
Main_topics['Steel'] = topic_var['5']
Main_topics['Masonry'] = 0.01*topic_var['1']+ 0.1*topic_var['17']
Main_topics['Lowrise'] = topic_var['10']
Main_topics['Medrise'] = 0.13*topic_var['0']
Main_topics['Regional'] = topic_var['1']
Main_topics['Urban'] = topic_var['15']
Main_topics['Residential'] = topic_var['16']


In [32]:
Main_topics

,id,Earthquake,Flood,Tsunami,Hurricane,RC,Wood,Steel,Masonry,Lowrise,Medrise,Regional,Urban,Residential
0,17496,0.7331,0.0000,0.0000,0.000000,0.051317,0.0000,0.0000,0.073784,0.0,0.000000,0.0474,0.0000,0.0000
1,15770,0.0749,0.8425,0.0000,0.000000,0.005243,0.0000,0.0000,0.007885,0.0,0.000000,0.0395,0.0000,0.0142
2,20395,0.6605,0.0000,0.0000,0.000000,0.046235,0.0000,0.0000,0.066881,0.0,0.000000,0.0831,0.2085,0.0000
3,19715,0.1495,0.3936,0.0000,0.000000,0.010465,0.0000,0.0000,0.015060,0.0,0.000000,0.0110,0.2341,0.1719
4,21380,0.7865,0.0817,0.0000,0.055400,0.055055,0.0000,0.0157,0.078650,0.0,0.000000,0.0000,0.0318,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,18286,0.7514,0.0000,0.0000,0.059500,0.052598,0.0000,0.0000,0.076037,0.0,0.000000,0.0897,0.0000,0.0215
1134,18075,0.6884,0.0220,0.0000,0.013363,0.064088,0.0161,0.0000,0.069846,0.0,0.002067,0.1006,0.0947,0.0000
1135,18026,0.6666,0.0000,0.0000,0.000000,0.153762,0.0000,0.0638,0.066972,0.0,0.013923,0.0312,0.0000,0.0000
1136,21170,0.8803,0.0000,0.0000,0.000000,0.061621,0.0000,0.0000,0.088976,0.0,0.000000,0.0946,0.0000,0.0000


In [33]:
topics = ['Earthquake','Flood','Tsunami','Hurricane','RC','Wood','Steel','Masonry','Lowrise','Medrise','Regional','Urban','Residential']
#Main_topics['Govern_Topic'] = Main_topics.idxmax(axis = 1)

tempdf = Main_topics.drop(['id'], axis =1)

Main_topics['Govern_Top'] = tempdf.idxmax(axis = 1)

tempdf = Main_topics.drop(['id','RC','Wood','Steel','Masonry','Lowrise','Medrise','Regional','Urban','Residential','Govern_Top'], axis =1)
Main_topics['Hazard'] = tempdf.idxmax(axis = 1)





In [27]:
Main_topics.to_csv('temp.csv', index=False)

In [34]:
Main_topics['Material'] = " "
Main_topics['Type'] = " "
Main_topics['Locationwise'] = " "
Main_topics['Usage'] = " "
tempdf = Main_topics.filter(['RC','Wood','Steel','Masonry'],axis = 1)
for row in Main_topics.index:
    if max(tempdf.iloc[row]) >= 0.0808:
        Main_topics['Material'].iloc[row] = tempdf.iloc[row].idxmax()
        
tempdf = Main_topics.filter(['Lowrise','Medrise'],axis = 1)
for row in Main_topics.index:
    if max(tempdf.iloc[row]) >= 0.024:
        Main_topics['Type'].iloc[row] = tempdf.iloc[row].idxmax()
        
tempdf = Main_topics.filter(['Regional','Urban'],axis = 1)
for row in Main_topics.index:
    if max(tempdf.iloc[row]) >= 0.0717:
        Main_topics['Locationwise'].iloc[row] = tempdf.iloc[row].idxmax()

tempdf = Main_topics.filter(['Residential'],axis = 1)
for row in Main_topics.index:
    if max(tempdf.iloc[row]) >= 0.05:
        Main_topics['Usage'].iloc[row] = tempdf.iloc[row].idxmax()
           


C:\Users\User\AppData\Local\Temp\ipykernel_14548\3991194342.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_topics['Material'].iloc[row] = tempdf.iloc[row].idxmax()
C:\Users\User\AppData\Local\Temp\ipykernel_14548\3991194342.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_topics['Type'].iloc[row] = tempdf.iloc[row].idxmax()
C:\Users\User\AppData\Local\Temp\ipykernel_14548\3991194342.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [38]:
Main_topics['Keywords'] = df_merged2['Keywords']
Main_topics['Approach'] = df_merged2['Method']

Main_topics

In [40]:
Main_topics.to_csv('FinalOutTopics.csv', index=False)